In [1]:
import os
import torch
from sklearn.model_selection import train_test_split

# Imports de la libreria propia
from vecopsciml.kernels.derivative import DerivativeKernels
from vecopsciml.utils import TensOps

# Imports de las funciones creadas para este programa
from models.non_constant_diffusivity import NonConstantDiffusivityNeuralNetwork
from utils.folders import create_folder
from utils.load_data import load_data
from trainers.train import train_loop

In [2]:
# Creamos los paths para las distintas carpetas
ROOT_PATH = r'C:\Users\usuario\Desktop\rmunozTMELab\Physically-Guided-Machine-Learning'
DATA_PATH = os.path.join(ROOT_PATH, r'data\non_linear\non_linear.pkl')
RESULTS_FOLDER_PATH = os.path.join(ROOT_PATH, r'results\non_linear')
MODEL_RESULTS_PATH = os.path.join(ROOT_PATH, r'results\non_linear\modelo_prueba_2')

# Creamos las carpetas que sean necesarias (si ya están creadas se avisará de ello)
create_folder(RESULTS_FOLDER_PATH)
create_folder(MODEL_RESULTS_PATH)

Folder already exists at: C:\Users\usuario\Desktop\rmunozTMELab\Physically-Guided-Machine-Learning\results\non_linear
Folder already exists at: C:\Users\usuario\Desktop\rmunozTMELab\Physically-Guided-Machine-Learning\results\non_linear\modelo_prueba_2


In [3]:
# Load dataset
dataset = load_data(DATA_PATH)

Data successfully loaded from: C:\Users\usuario\Desktop\rmunozTMELab\Physically-Guided-Machine-Learning\data\non_linear\non_linear.pkl


In [4]:
# Convolutional filters to derivate
dx = dataset['x_step_size']
dy = dataset['y_step_size']
D = DerivativeKernels(dx, dy, 0).grad_kernels_two_dimensions()

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [6]:
# Tratamiento de los datos para dividirlos en train y test
X_train = torch.Tensor(dataset['X_train']).unsqueeze(1)
y_train = TensOps(torch.tensor(dataset['y_train'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
K_train = TensOps(torch.tensor(dataset['k_train'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
f_train = TensOps(torch.tensor(dataset['f_train'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)

X_val = torch.Tensor(dataset['X_val']).unsqueeze(1)
y_val = TensOps(torch.tensor(dataset['y_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
K_val = TensOps(torch.tensor(dataset['k_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
f_val = TensOps(torch.tensor(dataset['f_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)

X_np = X_train
y_np = y_train.values
K_np = K_train.values
f_np = f_train.values

X_train_np, X_test_np, y_train_np, y_test_np, K_train_np, K_test_np, f_train_np, f_test_np = train_test_split(X_np, y_np, K_np, f_np, test_size=0.2, random_state=42)

X_train = X_train_np.to(device)
X_test = X_test_np.to(device)

y_train = TensOps(y_train_np.to(device), space_dimension=y_train.space_dim, contravariance=0, covariance=0)
y_test = TensOps(y_test_np.to(device), space_dimension=y_train.space_dim, contravariance=0, covariance=0)

K_train = TensOps(K_train_np.to(device), space_dimension=K_train.space_dim, contravariance=0, covariance=0)
K_test = TensOps(K_test_np.to(device), space_dimension=K_train.space_dim, contravariance=0, covariance=0)

f_train = TensOps(f_train_np.to(device), space_dimension=K_train.space_dim, contravariance=0, covariance=0)
f_test = TensOps(f_test_np.to(device), space_dimension=K_train.space_dim, contravariance=0, covariance=0)

In [7]:
# Arquitectura del modelo
input_shape = X_train[0].shape  # [1, 10, 8]
hidden1_dim = 20
hidden2_dim = 20
output_shape = y_train.values[0].shape  # [1, 10, 10]

In [8]:
# Se carga el modelo y el optimizador
model = NonConstantDiffusivityNeuralNetwork(input_shape, hidden1_dim, hidden2_dim, output_shape)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [17]:
# Parametros de entrenamiento
start_epoch = 1800
n_epochs = 2500

batch_size = 64
n_checkpoints = 10

second_lr = 5e-4

train_loop(model, optimizer, n_checkpoints,
           X_train, y_train, X_test, y_test, f_train, f_test,
           D=D, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PATH, new_lr=second_lr
        )

Start training
Starting from a checkpoint. Epoch 1800.
Epoch 1800, Train loss: 3.671e+04, Test loss: 7.756e+04, MSE(e): 1.499e-03, MSE(pi1): 6.928e-02, MSE(pi2): 1.371e-03, MSE(pi3): 1.479e-01
Epoch 1900, Train loss: 3.182e+04, Test loss: 7.484e+04, MSE(e): 1.440e-03, MSE(pi1): 1.026e-01, MSE(pi2): 1.340e-03, MSE(pi3): 7.163e-02
Epoch 2000, Train loss: 3.211e+04, Test loss: 8.282e+04, MSE(e): 1.455e-03, MSE(pi1): 1.018e-01, MSE(pi2): 1.323e-03, MSE(pi3): 7.374e-02
Epoch 2100, Train loss: 2.985e+04, Test loss: 6.858e+04, MSE(e): 1.445e-03, MSE(pi1): 8.100e-02, MSE(pi2): 1.318e-03, MSE(pi3): 7.292e-02
Epoch 2200, Train loss: 3.429e+04, Test loss: 7.411e+04, MSE(e): 1.430e-03, MSE(pi1): 1.291e-01, MSE(pi2): 1.319e-03, MSE(pi3): 7.073e-02
Epoch 2300, Train loss: 2.893e+04, Test loss: 6.308e+04, MSE(e): 1.458e-03, MSE(pi1): 7.300e-02, MSE(pi2): 1.317e-03, MSE(pi3): 7.044e-02
Epoch 2400, Train loss: 3.006e+04, Test loss: 6.341e+04, MSE(e): 1.400e-03, MSE(pi1): 9.125e-02, MSE(pi2): 1.312e-03,

In [15]:
# Parametros de entrenamiento
start_epoch = 1800
n_epochs = 3000

batch_size = 64
n_checkpoints = 10

second_lr = 5e-4

train_loop(model, optimizer, n_checkpoints,
           X_train, y_train, X_test, y_test, f_train, f_test,
           D=D, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PATH, new_lr=second_lr
        )

Start training
Starting from a checkpoint. Epoch 1800.
Epoch 1800, Train loss: 1.080e+08, Test loss: 1.241e+08, MSE(e): 6.612e+00, MSE(pi1): 4.150e+02, MSE(pi2): 3.031e+00, MSE(pi3): 3.501e+00
Epoch 1900, Train loss: 2.838e+05, Test loss: 4.233e+05, MSE(e): 7.242e-03, MSE(pi1): 6.938e-02, MSE(pi2): 4.333e-03, MSE(pi3): 2.044e+00
Epoch 2000, Train loss: 2.041e+05, Test loss: 2.903e+05, MSE(e): 4.986e-03, MSE(pi1): 8.663e-02, MSE(pi2): 3.260e-03, MSE(pi3): 1.455e+00
Epoch 2100, Train loss: 1.590e+05, Test loss: 2.280e+05, MSE(e): 3.883e-03, MSE(pi1): 3.914e-01, MSE(pi2): 2.539e-03, MSE(pi3): 8.101e-01


KeyboardInterrupt: 

In [11]:
# # Parametros de entrenamiento
# start_epoch = 36000
# n_epochs = 60000

# batch_size = 64
# n_checkpoints = 10

# second_lr = 1e-5

# train_loop(model, optimizer, n_checkpoints,
#            X_train, y_train, X_test, y_test, D=D, 
#            start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
#            model_results_path=MODEL_RESULTS_PATH, new_lr=second_lr
#         )